In [38]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
import ast

In [137]:
df_rotamers=pd.read_csv("df_rotamer.csv")
df_rotamers.set_index('pdb_id',inplace=True)
resi = df_rotamers["bound_residue"][0]
r = ast.literal_eval(resi)
len(df_rotamers)
df_rotamers.head()
for i in range(len(df_rotamers)):
    np.array(zeros
    

/tmp/ipykernel_70075/1146157281.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  resi = df_rotamers["bound_residue"][0]


2239

In [59]:
x = df_rotamers
x = x.drop(columns=['metal','ion','pdb','pdb_id'])
y = df_rotamers["metal"]

x.head()
motif_binding_array = x["motif_binding_xyz"]
motif_binding_array[0]
dic = ast.literal_eval(motif_binding_array[0])

#x_train,x_test,y_train_y_test = train_test_split(x,y,test_size=.1)

ValueError: malformed node or string on line 1: <ast.Call object at 0x7f55f8a975b0>

test_string = '{"Nikhil" : 1, "Akshat" : 2, "Akash" : 3}'
res = ast.literal_eval(test_string)
motif_

In [55]:
string = motif_binding_array[0]
string = string.replace(" array"," ")
string
#dict=ast.literal_eval(string)

"{'xyz': ([[[-1.849, -6.217,  5.614],\n        [-1.722, -5.22 ,  4.553],\n        [-0.399, -4.468,  4.672],\n        [ 0.209, -4.082,  3.667],\n        [-2.889, -4.227,  4.63 ],\n        [-2.819, -3.114,  3.601],\n        [-2.095, -1.942,  3.848],\n        [-3.494, -3.226,  2.386],\n        [-2.053, -0.902,  2.903],\n        [-3.459, -2.201,  1.439],\n        [-2.744, -1.043,  1.705],\n        [-2.759, -0.016,  0.789],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan]]], dtype=float32), 'mask': ([[ True,  True,  True,  True,  True,  True,  True,  True,  True,\n         True,  True,  True, False, False, False, False, False, 

In [64]:
mdf =pd.MultiIndex.from_frame(df_rotamers)
print(mdf)

MultiIndex([('1WWS_1.pdb', '1WWS', 'CA', 'Ca²⁺', ...),
            ('4K9N_1.pdb', '4K9N', 'MG', 'Mg²⁺', ...),
            ('2A0Q_1.pdb', '2A0Q',  'K',   'K⁺', ...),
            ('3E7Z_1.pdb', '3E7Z', 'ZN', 'Zn²⁺', ...),
            ('7MCI_1.pdb', '7MCI', 'CA', 'Ca²⁺', ...),
            ('4RF4_1.pdb', '4RF4', 'MG', 'Mg²⁺', ...),
            ('8AE1_1.pdb', '8AE1', 'CA', 'Ca²⁺', ...),
            ('7TSC_1.pdb', '7TSC', 'ZN', 'Zn²⁺', ...),
            ('4CBB_1.pdb', '4CBB',  'K',   'K⁺', ...),
            ('4KCP_1.pdb', '4KCP', 'ZN', 'Zn²⁺', ...),
            ...
            ('7TSP_1.pdb', '7TSP', 'ZN', 'Zn²⁺', ...),
            ('2B00_1.pdb', '2B00', 'CA', 'Ca²⁺', ...),
            ('1L5B_1.pdb', '1L5B',  nan,  'Na⁺', ...),
            ('3T4D_3.pdb', '3T4D',  'K',   'K⁺', ...),
            ('6R7D_1.pdb', '6R7D', 'NI', 'Ni²⁺', ...),
            ('1C3H_1.pdb', '1C3H', 'CA', 'Ca²⁺', ...),
            ('7RWX_1.pdb', '7RWX', 'ZN', 'Zn²⁺', ...),
            ('6PN8_1.pdb', '6PN8', 'ZN', 'Zn²⁺', 

In [71]:
df_homo_filter = pd.read_csv("df_homo_filter.csv")
pdbs = df_homo_filter["pdb_id"].unique()
df_homo_filter.set_index('pdb_id',inplace=True)
CategoricalIndex

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative,metal_binding_id,metal
pdb_id,,,,,,,,,,,,,
5LDB,5LDB-MG-1,MG,Mg²⁺,T2,THR.,HOH.THR.,True,2,3,2,True,1,MG
2SHK,2SHK-MG-1,MG,Mg²⁺,A1G2R2,ALA.ARG.GLY.,ALA.GLN.HOH.,True,5,5,2,False,1,MG
2R6T,2R6T-MG-1,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,1,MG
2R6T,2R6T-MG-2,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2,MG
2R6T,2R6T-MG-3,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,3,MG
